In [2]:
!pip install sentence-transformers faiss-cpu transformers

  Using cached faiss_cpu-1.12.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.0 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline

In [13]:
llm = pipeline("text2text-generation", model="google/flan-t5-large")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


# Prueba sin RAG
Se llama al LLM sin ninguna modificación y se observa que alucina.

In [14]:
question = "Quién ganó la Media Maratón de Buenos Aires 2025?"
result_no_context = llm(question, max_new_tokens=50, do_sample=False)[0]['generated_text']
print("Sin contexto:", result_no_context)

Sin contexto: antonio gonzález


# Configurar el RAG

## 0. Importar los documentos a utilizar
Se utilizan directamente los textos de dos noticias para hacer el ejemplo más fácil de entender y, además se utilizan contextos distintos para verificar la capacidad de poder buscvar semánticamente lo más parecido a lo que se pide en la consulta.

In [16]:
#Link: https://www.infobae.com/deportes/2025/08/24/la-media-maraton-de-buenos-aires-se-celebra-hoy-con-un-record-de-mas-de-27000-participantes/
noticia_1 = """
Más de 27.000 corredores participaron este domingo 24 de agosto en la Media Maratón de Buenos Aires 2025, que tuvo su largada y llegada en la intersección de Avenida Figueroa Alcorta y Dorrego, en el barrio de Palermo. El evento, que comenzó a las 8:00, se consolidó como la prueba atlética más convocante de Latinoamérica, con un récord de inscriptos de más de 30 países y la presencia de figuras de atletismo de élite internacional.

El recorrido de los 21 kilómetros llevó a los participantes por un circuito urbano que atravesó Palermo y se extendió por los barrios del norte y centro de la ciudad, antes de regresar al punto de partida. El trazado, certificado y homologado por la World Athletics, garantizó condiciones técnicas de primer nivel y fue escenario de marcas históricas en ediciones anteriores.

El primer puesto fue para Jacob Kiplimo: el ugandés logró romper el récord del circuito, con 58:29 minutos.

El segundo lugar fue para Seifu Tura Abdiwak, de Etiopía, con un registro de 59 minutos y 56 segundos. El tercer escalón del podio lo ocupó el keniano Vinicent Nyamongo Nyageo, que completó la media maratón en 59 minutos y 57 segundos.

En cuanto a los argentinos, el podio masculino nacional quedó conformado del siguiente modo: Laureano Rosa se consagró campeón argentino con un tiempo de 1 hora, 4 minutos y 8 segundos. Lo siguió Manuel Córsico, quien completó el recorrido en 1 hora, 5 minutos y 1 segundo. El tercer puesto fue para Nahuel Di Leva, que registró 1 hora, 5 minutos y 14 segundos.

En la rama femenina internacional, el primer puesto fue para Verónica Loleo (Kenia), quien alcanzó la meta en 1 hora, 6 minutos y 58 segundos. En segundo lugar quedó Ftaw Zeray Bezabh, con un tiempo de 1 hora, 7 minutos y 7 segundos, y el tercer puesto lo ocupó Catherine Reline Amanong’ole, también de Kenia, con 1 hora, 7 minutos y 13 segundos.

En la clasificación argentina, se destacó la marplatense Florencia Borelli, quien cruzó la meta en 1 hora, 9 minutos y 21 segundos y batió el récord sudamericano y argentino de la distancia.
"""

# link: https://www.infobae.com/sociedad/2025/08/24/clima-primaveral-en-el-amba-y-alertas-por-lluvias-para-ocho-provincias-el-pronostico-del-tiempo-para-el-resto-de-la-semana/
noticia_2 = """
De cara a lo que resta de la semana, el Servicio Meteorológico Nacional (SMN) pronosticó que el Área Metropolitana de Buenos Aires (AMBA) registrará días soleados y con ocasional nubosidad variable, aunque sin presencia de lluvias. No obstante, el organismo estatal emitió alertas por precipitaciones para ocho provincias.

Para esta mañana, el SMN confirmó que el cielo estará completamente despejado, con una mínima de 8°C. Por la tarde, el cielo tendrá apenas unas pocas nubes, pero soplarán ráfagas de viento entre los 42 y los 50 kilómetros por hora, con una temperatura de 18°C. Durante la noche, las ráfagas se mantendrán y habrá un descenso a los 12°C. Vientos del noroeste.

El lunes estará completamente despejado durante toda la jornada, además de que será el día donde las máximas empezarán a subir de a poco. La madrugada comenzará fresca por sus 9°C, que ascenderá a un grado durante la mañana. Por la tarde, la suba marcará los 20°C, mientras que a la noche bajará a los 14°C.

Las condiciones se replicarán para el martes, sin nubes en el firmamento y la temperatura oscilará entre los 9°C de mínima y los 20°C de máxima. Desde el miércoles hacia el viernes, la nubosidad irá de algo a parcialmente nublado. Además, el viernes será el día más caluroso, ya que se prevé una máxima de 22°C. El sábado tendrá un cielo parcialmente nublado, que cambiará a nublado para la tarde y una mínima chance de precipitaciones del 10%.
"""

## 1. Dividir los documentos en chunks
Los documentos se dividen en pedacitos para que puedan ser correctamente procesados.

In [56]:
import math

def split_text_recursive(text, max_length=300, overlap=50, separators=["\n\n", "\n", ". "]):
    chunks = []

    def recursive_split(segment):
        if len(segment) <= max_length:
            chunks.append(segment.strip())
            return
        for sep in separators:
            if sep in segment:
                parts = segment.rsplit(sep, 1)
                if len(parts[0]) <= max_length:
                    chunks.append(parts[0].strip())
                    recursive_split(parts[1])
                    return
        # Si no hay separador, cortar de manera forzada
        chunks.append(segment[:max_length].strip())
        recursive_split(segment[max_length - overlap:])

    recursive_split(text)
    return chunks

In [74]:
all_news = [noticia_1, noticia_2]
chunks = []
for news in all_news:
    chunks.extend(split_text_recursive(news, max_length=455))

print(f"Cantidad de chunks: {len(chunks)}")

Cantidad de chunks: 9


## 2. Generar los embeddings
Estos embeddings son la representacióm numérica de la información. En este ejemplo se utiliza Faiss para almacenar los vectores y luego poder hacer búsquedas rápidas.

In [75]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [76]:
embeddings = model.encode(chunks, show_profress_bar=True)
dim = embeddings.shape[1]
print(f"Embedding size: {embeddings.shape}, Dimension: {dim}")

Embedding size: (9, 384), Dimension: 384


In [82]:
index = faiss.IndexFlatL2(dim) # L2 = euclidian distance (menor distancia = más cercano / más parecido semánticamente)
index.add(np.array(embeddings, dtype="float32"))
print(f"Vectors quantity: {index.ntotal}")

Vectors quantity: 9


## 3. Buscar el chunk más relevante
En este caso estamos buscando las mejores 3 coincidencias

In [79]:
query = "Quién ganó la Media Maratón de Buenos Aires 2025?"
query_vec = model.encode([query])
distances, records_index = index.search(np.array(query_vec, dtype="float32"), k=3)
top_chunks = [chunks[i] for i in records_index[0]]
print(top_chunks)

['Más de 27.000 corredores participaron este domingo 24 de agosto en la Media Maratón de Buenos Aires 2025, que tuvo su largada y llegada en la intersección de Avenida Figueroa Alcorta y Dorrego, en el barrio de Palermo. El evento, que comenzó a las 8:00, se consolidó como la prueba atlética más convocante de Latinoamérica, con un récord de inscriptos de más de 30 países y la presencia de figuras de atletismo de élite internacional.\n\nEl recorrido de lo', 's.\n\nEl primer puesto fue para Jacob Kiplimo: el ugandés logró romper el récord del circuito, con 58:29 minutos.\n\nEl segundo lugar fue para Seifu Tura Abdiwak, de Etiopía, con un registro de 59 minutos y 56 segundos. El tercer escalón del podio lo ocupó el keniano Vinicent Nyamongo Nyageo, que completó la media maratón en 59 minutos y 57 segundos.\n\nEn cuanto a los argentinos, el podio masculino nacional quedó conformado del siguiente modo: Laureano Ros', 'En segundo lugar quedó Ftaw Zeray Bezabh, con un tiempo de 1 hora, 7 minu

## 4. Generando la respuesta conm un LLM

In [84]:
context = "\n".join(top_chunks)

prompt = f"""
      Eres un bot servicial e informativo que responde preguntas utilizando el texto referencia incluido a continuación.
      Asegúrate de responder en una oración completa, siendo exhaustivo y proporcionando toda la información de fondo relevante.
      Sin embargo, estás hablando con una audiencia no técnica, por lo que debes desglosar los conceptos complicados y mantener un tono amigable y conversacional.
      Por favor, responde en el idioma de la pregunta.

      PREGUNTA: '{question}'
      TEXTO DE REFERENCIA: '{context}'
      RESPUESTA:
      """

In [85]:
result_with_context = llm(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']
print("Con contexto:", result_with_context)

Con contexto: El primer puesto fue para Jacob Kiplimo: el ugandés logró romper el récord del circuito, con 58:29 minutos. El segundo lugar fue para Seifu Tura Abdiwak, de Etiopa, con un registro de 59 minutos y 56 segundos. El tercer 
